In [1]:
%reload_ext autoreload
%autoreload 2

import os
import argparse
import sys
import time
import cv2

from joblib import Parallel, delayed

sys.path.append(os.path.join(os.environ['REPO_DIR'], 'utilities'))
from utilities2015 import *

from matplotlib.path import Path
%matplotlib inline

import numpy as np

In [2]:
features_dir = '/home/yuncong/csd395/CSHL_patch_features_Sat16ClassFinetuned/'
# train_features_dir = features_dir + '/train'
# train_features_dir = features_dir + '/train_v2'
train_features_dir = features_dir + '/train_v3'

In [3]:
labels = ['BackG', '5N', '7n', '7N', '12N', 'Pn', 'VLL', 
          '6N', 'Amb', 'R', 'Tz', 'RtTg', 'LRt', 'LC', 'AP', 'sp5']

labels_index = dict((j, i) for i, j in enumerate(labels))

labels_from_surround = dict( (l+'_surround', l) for l in labels[1:])

labels_surroundIncluded_list = labels[1:] + [l+'_surround' for l in labels[1:]]
labels_surroundIncluded = set(labels_surroundIncluded_list)

labels_surroundIncluded_index = dict((j, i) for i, j in enumerate(labels_surroundIncluded_list))

colors = np.random.randint(0, 255, (len(labels_index), 3))

In [4]:
from sklearn.svm import SVC
from sklearn.externals import joblib

In [5]:
# svm_dir = 'svm_Sat16ClassFinetuned'
# svm_dir = 'svm_Sat16ClassFinetuned_v2'
svm_dir = 'svm_Sat16ClassFinetuned_v3'
create_if_not_exists(svm_dir)

In [7]:
# train svm classifiers
stack = 'MD589'

t = time.time()

for label in labels[1:]:
            
    print label
    
    features_allClasses_pos = load_hdf(train_features_dir + '/%(stack)s_%(label)s_features.hdf' % {'stack': stack, 'label': label})
    features_allClasses_neg = load_hdf(train_features_dir + '/%(stack)s_%(label)s_surround_features.hdf' % {'stack': stack, 'label': label})

    train_data = np.r_[features_allClasses_pos, features_allClasses_neg]
    train_labels = np.r_[np.ones((features_allClasses_pos.shape[0], )), 
                         -np.ones((features_allClasses_neg.shape[0], ))]
    
    svc = SVC(C=1.0, kernel='rbf', degree=3, gamma='auto', coef0=0.0, shrinking=True, 
          probability=True, tol=0.001, cache_size=200, class_weight=None, verbose=False, 
          max_iter=-1, decision_function_shape=None, random_state=None)
    
    svc.fit(train_data, train_labels)
    
    joblib.dump(svc, svm_dir + '/%(label)s_svm.pkl' % {'label': label})
    
    del features_allClasses_pos, features_allClasses_neg, train_data, train_labels, svc
    
sys.stderr.write('svm train: %.2f seconds\n' % (time.time() - t)) # ~41s v3

5N
7n
7N
12N
Pn
VLL
6N
Amb
R
Tz
RtTg
LRt
LC
AP
sp5


svm train: 41.19 seconds


In [8]:
# Load pre-computed svm classifiers

svc_allClasses = {}
for label_ind, label in enumerate(labels[1:]):
    svc_allClasses[label] = joblib.load(svm_dir + '/%(label)s_svm.pkl' % {'label': label})

In [16]:
# output

# predictions_rootdir = '/oasis/projects/nsf/csd395/yuncong/CSHL_patch_predictions_svm_Sat16ClassFinetuned'
# predictions_rootdir = '/oasis/projects/nsf/csd395/yuncong/CSHL_patch_predictions_svm_Sat16ClassFinetuned_v2'
predictions_rootdir = '/oasis/projects/nsf/csd395/yuncong/CSHL_patch_predictions_svm_Sat16ClassFinetuned_v3'
create_if_not_exists(predictions_rootdir)

'/oasis/projects/nsf/csd395/yuncong/CSHL_patch_predictions_svm_Sat16ClassFinetuned_v3'

In [17]:
def svm_predict(stack, sec):
    
    test_features_dir = features_dir + '/%(stack)s/%(sec)04d' % {'stack': stack, 'sec': sec}
#     features_roi = bp.unpack_ndarray_file(test_features_dir + '/%(stack)s_%(sec)04d_features.bp' % {'stack': stack, 'sec': sec})
    features_roi = load_hdf(test_features_dir + '/%(stack)s_%(sec)04d_roi1_features.hdf' % {'stack': stack, 'sec': sec})
    n = features_roi.shape[0]

    predictions_dir = predictions_rootdir + '/%(stack)s/%(sec)04d' % {'stack': stack, 'sec': sec}
    create_if_not_exists(predictions_dir)

###### all labelmaps in one file #########

#     probs = np.zeros((n, len(labels)-1))
#     for label_ind, label in enumerate(labels[1:]):
#         svc = svc_allClasses[label]
#         probs[:, label_ind] = svc.predict_proba(features_roi)[:, svc.classes_.tolist().index(1.)]
#     np.save(predictions_dir + '/%(stack)s_%(sec)04d_roi1_scores.npy' % {'stack': stack, 'sec': sec}, probs)
    
###### separate labelmap in different files #######

    for label_ind, label in enumerate(labels[1:]):
        svc = svc_allClasses[label]
        probs = svc.predict_proba(features_roi)[:, svc.classes_.tolist().index(1.)]
        np.save(predictions_dir + '/%(stack)s_%(sec)04d_roi1_%(label)s_sparseScores.npy' % {'stack': stack, 'sec': sec, 'label': label}, 
                probs)

In [21]:
stack = 'MD594'
first_detect_sec, last_detect_sec = detect_bbox_range_lookup[stack]

t = time.time()
Parallel(n_jobs=8)(delayed(svm_predict)(stack=stack, sec=sec) 
                   for sec in range(first_detect_sec, last_detect_sec+1))

sys.stderr.write('svm predict: %.2f seconds\n' % (time.time() - t)) # 2412 s = 40 min / stack

svm predict: 2412.87 seconds


In [ ]:
t = time.time()
for sec in range(first_detect_sec, last_detect_sec+1, 10):
    f(stack=stack, sec=sec)
print time.time() - t # ~92 s